In [1]:
import torch as th
import torch.nn as nn

/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
B = 1
H = 8
H_GW = 4

vis_feats = th.randn([1, H]); vis_feats.shape
aud_feats = th.randn([1, H]); aud_feats.shape
gw = th.randn([1, H_GW]); gw.shape
if H != H_GW:
  assert H > H_GW, f"Invalid vis-aud: {H} and GW: {H_GW} shape combination"
  # Pad the reduced global workspace
  gw = th.cat([
    gw,
    gw.new_zeros([B, H - H_GW])
  ], dim=-1)
  # print(f"Post cat dim: {gw.shape}")

  # Generate the key_padding_mask
  # NOTE: according to doc of attn_mask field in Pytorch, a True / 1 value mean
  # that the position is not allowed to attend to
  attn_mask = gw.new_zeros([B, 3, H]).bool()
  attn_mask[:, 2, H_GW:] = True

  # attn_mask.shape; attn_mask.float()

q = th.cat([
  vis_feats[:, None, :],
  aud_feats[:, None, :],
  gw[:, None, :]
], dim=1); q.shape

kv = th.cat([
  vis_feats[:, None, :],
  aud_feats[:, None, :],
  gw[:, None, :]
], dim=1); kv.shape
kv = th.cat([
  kv,
  kv.new_zeros([1, 1, H])
], dim=1); kv.shape

torch.Size([1, 4, 8])

In [91]:
mha = nn.MultiheadAttention(
  H,
  1,
  dropout=0.0,
  add_zero_attn=False,
  batch_first=True,
  kdim=H,
  vdim=H
)

# key_padding_mask = th.ones([3, H]); key_padding_mask.shape
attn_values, attn_weights = mha(q, kv, kv); attn_values.shape

torch.Size([1, 3, 8])

In [86]:
q.shape, kv.shape, kv.mT.shape
A = (q @ kv.mT * H ** -0.5).softmax(dim=-1); A.shape

torch.Size([1, 3, 4])

In [87]:
B = (A @ kv); B.shape

torch.Size([1, 3, 8])

In [92]:
attn_values, B

(tensor([[[-0.1607, -0.1777, -0.0145,  0.0072, -0.2937,  0.1897, -0.0494,
           -0.3013],
          [-0.1219, -0.1638,  0.0980, -0.0303, -0.2723,  0.1179, -0.0071,
           -0.2830],
          [-0.1423, -0.2004,  0.1438, -0.0354, -0.3020,  0.1045,  0.0021,
           -0.3100]]], grad_fn=<TransposeBackward0>),
 tensor([[[-0.6238,  0.4845, -0.1870,  0.2597, -0.6642, -0.6949,  1.1371,
           -0.1859],
          [-1.3688,  0.0429, -0.2187,  0.2357, -0.4113, -0.1630,  1.0966,
            0.9588],
          [-1.0530,  0.0263, -0.0991, -0.0427, -0.2852, -0.1889,  0.6492,
            0.3603]]]))